In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from gensim.models import Word2Vec, KeyedVectors
import matplotlib.pyplot as plt

In [2]:
# Load the data
file_path = 'BusinessNewsData.csv'
data = pd.read_csv(file_path)
data.head()

,Title,Description,Meta-Summary,Summary,syndicate-catagorys,Keywords-for-related-articles
0,টাকা ফেরতে ই–অরেঞ্জের লিখিত\r\nপ্রতিশ্রুতি চান...,ই–অরেঞ্জের মালিকপক্ষ যেন বাণিজ্য মন্ত্রণালয়ের ...,ভুক্তভোগী গ্রাহকদের পক্ষে চারজন আজ বুধবার সচিব...,ভুক্তভোগী গ্রাহকদের পক্ষে চারজন আজ বুধবার সচিব...,বাণিজ্য,"ই কমার্স, বাণিজ্য"
1,ক্লিংকার রপ্তানিতে শুল্ক বৃদ্ধির প্রস্তাব ভিয়ে...,সিমেন্ট তৈরির প্রধান কাঁচামাল ক্লিংকার রপ্তানি...,ভিয়েতনামের শুল্ক বাড়ানো হলে ক্লিংকারের দর আরেক...,ভিয়েতনামের শুল্ক বাড়ানো হলে ক্লিংকারের দর আরেক...,বাণিজ্য,"আমদানি রপ্তানি, সিমেন্ট খাত"
2,সূচক ও লেনদেন কমেছে শেয়ারবাজারে,সূচক ও লেনদেন কমেছে দেশের শেয়ারবাজারে। আজ বুধব...,সূচক ও লেনদেন কমেছে দেশের শেয়ারবাজারে। আজ বুধব...,সূচক ও লেনদেন কমেছে দেশের শেয়ারবাজারে। আজ বুধব...,বাণিজ্য,"ডিএসই, সিএসই, ডিএসইএক্স"
3,এমডির চলতি দায়িত্বে শিরীন আখতার,বাংলাদেশ কৃষি ব্যাংকের উপব্যবস্থাপনা পরিচালক শ...,বাংলাদেশ কৃষি ব্যাংকের উপব্যবস্থাপনা পরিচালক শ...,বাংলাদেশ কৃষি ব্যাংকের উপব্যবস্থাপনা পরিচালক শ...,NaN,ব্যাংক
4,মালা শাড়ির আনোয়ার হোসেন,"মালা শাড়ি, বাংলাদেশের শাড়ির জগতে প্রথম সুপরিচি...",দেশের অন্যতম বৃহৎ শিল্পগোষ্ঠী আনোয়ার গ্রুপের চ...,দেশের অন্যতম বৃহৎ শিল্পগোষ্ঠী আনোয়ার গ্রুপের চ...,NaN,"উদ্যোক্তা, ব্যবসায়ী, অর্থনীতি"


In [3]:
# Data Cleaning: Remove NaN values
data = data.dropna(subset=['Description', 'Keywords-for-related-articles'])

In [4]:
from bnlp import BengaliCorpus as corpus

# Define Bangla stopwords
bangla_stopwords=set(corpus.stopwords)

# Function for preprocessing text
def preprocess_bangla_text(text):
    # Normalize whitespace
    text = ' '.join(text.split())
    
    # Remove stopwords
    text = ' '.join(word for word in text.split() if word not in bangla_stopwords)
    
    # Define Bangla punctuation except full stop (।)
    bangla_punctuation = r'[!\"#$%&\'()*+,-./:;<=>?@\[\\\]^_`{|}~]–'
    
    # Remove all Bangla punctuation except full stop (।)
    text = re.sub(bangla_punctuation, '', text)
    
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    
    return text

In [5]:
# Apply preprocessing to the 'Description' column
data['preprocess_description'] = data['Description'].apply(preprocess_bangla_text)
print(data['preprocess_description'].head())

0    ই–অরেঞ্জের মালিকপক্ষ বাণিজ্য মন্ত্রণালয়ের কর্ম...
1    সিমেন্ট তৈরির প্রধান কাঁচামাল ক্লিংকার রপ্তানি...
2    সূচক লেনদেন কমেছে দেশের শেয়ারবাজারে। বুধবার চল...
3    বাংলাদেশ কৃষি ব্যাংকের উপব্যবস্থাপনা পরিচালক শ...
4    মালা শাড়ি, বাংলাদেশের শাড়ির জগতে সুপরিচিত ব্র্...
Name: preprocess_description, dtype: object


In [6]:
# Function for preprocessing text
from bnlp import CleanText

clean_text = CleanText(
   fix_unicode=True,
   unicode_norm=True,
   unicode_norm_form="NFKC",
   remove_url=True,
   remove_email=True,
   remove_emoji=True,
   remove_number=False,
   remove_digits=False,
   remove_punct=False,
   replace_with_url="",
   replace_with_email="<EMAIL>",
   replace_with_number="<NUMBER>",
   replace_with_digit="<DIGIT>",
   replace_with_punct = ""
)

data['clean_description'] = data['preprocess_description'].apply(clean_text)
print("Clean descriptions:")
print(data['clean_description'].head(20))

Clean descriptions:
0     ই–অরেঞ্জের মালিকপক্ষ বাণিজ্য মন্ত্রণালয়ের কর্...
1     সিমেন্ট তৈরির প্রধান কাঁচামাল ক্লিংকার রপ্তানি...
2     সূচক লেনদেন কমেছে দেশের শেয়ারবাজারে। বুধবার চ...
3     বাংলাদেশ কৃষি ব্যাংকের উপব্যবস্থাপনা পরিচালক শ...
4     মালা শাড়ি, বাংলাদেশের শাড়ির জগতে সুপরিচিত ব্...
5     তালেবান কাবুলের নিয়ন্ত্রণ করতেই প্রেসিডেন্ট আ...
6     রাজধানীর আগারগাঁও এলাকায় একসময় বস্তি, খালি জ...
7     রাজশাহীর গোদাগাড়ীতে ২০১৮ সালে দেড় শ টাকা খরচ...
8     ই-কমার্স প্রতিষ্ঠান ই–অরেঞ্জের মালিকপক্ষ প্রতা...
9     পর্যটন খাতের ব্যবসায়ীরা টিকে থাকতে চান। গ্রাহ...
10    সময়মতো ঋণের কিস্তি পরিশোধ নারী উদ্যোক্তারা ১ ...
11    দেশের অন্যতম বৃহৎ শিল্পগোষ্ঠী আনোয়ার গ্রুপের ...
12    গত রোববার তালেবান আফগানিস্তানের রাজধানী কাবুলে...
13    'ডিজিটাল মুদ্রা' শব্দটি শুনলে মানুষের চোখের সা...
14    সপ্তাহ হওয়া তালেবানের আফগানিস্তান নিয়ন্ত্রণে...
15    আফগানিস্তানের ভঙ্গুর অর্থনীতি এগিয়ে চলেছে মূল...
16    ৬৩ ধরনের পণ্য পচনশীল হিসেবে গণ্য হবে। জাতীয় র...
17    ক্ষুদ্র কৃষক উন্নয়ন ফ

In [7]:
# Tokenize the cleaned descriptions
from bnlp import BasicTokenizer

tokenizer = BasicTokenizer()
data['tokens'] = data['clean_description'].apply(tokenizer.tokenize)
print(data['tokens'].head())

0    [ই, –, অরেঞ্জের, মালিকপক্ষ, বাণিজ্য, মন্ত্রণাল...
1    [সিমেন্ট, তৈরির, প্রধান, কাঁচামাল, ক্লিংকার, র...
2    [সূচক, লেনদেন, কমেছে, দেশের, শেয়ারবাজারে, ।, ...
3    [বাংলাদেশ, কৃষি, ব্যাংকের, উপব্যবস্থাপনা, পরিচ...
4    [মালা, শাড়ি, ,, বাংলাদেশের, শাড়ির, জগতে, সুপ...
Name: tokens, dtype: object


In [16]:
from bangla_stemmer.stemmer import stemmer

# Initialize BanglaStemmer
stemmer = stemmer.BanglaStemmer()

# Function to tokenize by Bangla full stop (।) and create sentences
def tokenize_to_sentences(text):
    # Split by Bangla full stop (।)
    sentences = text.split('।')
    # Remove any empty strings or spaces from the resulting list
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    return sentences

# Apply tokenization to 'stemmed_description' column
data['sentences'] = data['clean_description'].apply(tokenize_to_sentences)

print(data['sentences'].head(10))

0    [ই–অরেঞ্জের মালিকপক্ষ বাণিজ্য মন্ত্রণালয়ের কর...
1    [সিমেন্ট তৈরির প্রধান কাঁচামাল ক্লিংকার রপ্তান...
2    [সূচক লেনদেন কমেছে দেশের শেয়ারবাজারে, বুধবার ...
3    [বাংলাদেশ কৃষি ব্যাংকের উপব্যবস্থাপনা পরিচালক ...
4    [মালা শাড়ি, বাংলাদেশের শাড়ির জগতে সুপরিচিত ব...
5    [তালেবান কাবুলের নিয়ন্ত্রণ করতেই প্রেসিডেন্ট ...
6    [রাজধানীর আগারগাঁও এলাকায় একসময় বস্তি, খালি ...
7    [রাজশাহীর গোদাগাড়ীতে ২০১৮ সালে দেড় শ টাকা খর...
8    [ই-কমার্স প্রতিষ্ঠান ই–অরেঞ্জের মালিকপক্ষ প্রত...
9    [পর্যটন খাতের ব্যবসায়ীরা টিকে থাকতে চান, গ্রা...
Name: sentences, dtype: object


In [22]:
import pandas as pd
import re

# Assuming 'data' and 'sentences_data' are already defined from the previous steps

# Function to check if a sentence contains digits
def contains_digits(sentence):
    return bool(re.search(r'[০-৯0-9]', sentence))  # Match Bangla or English digits

# Apply the function to each sentence in 'sentences_data'
sentences_data['contains_digits'] = sentences_data['Sentence'].apply(contains_digits)

# Filter sentences that contain digits
sentences_with_digits = sentences_data[sentences_data['contains_digits']]

# Count occurrences of each sentence
sentence_counts = sentences_with_digits['Sentence'].value_counts()

# Create a DataFrame with sentences and their frequencies
sentences_with_frequencies = pd.DataFrame({'sentence': sentence_counts.index, 'frequency': sentence_counts.values})

# Display the top 20 sentences containing numbers along with their frequencies
print("Sentences with digits and their frequencies:")
print(sentences_with_frequencies.head(50))

# Export sentences with digits and their frequencies to a CSV file
output_file_path = 'sentences_with_digits.csv'
sentences_with_frequencies.to_csv(output_file_path, index=False, encoding='utf-8-sig')

Sentences with digits and their frequencies:
                                             sentence  frequency
0                                দফায় কমে ৪ ৯৫৮ টাকা          5
1             ৪ জুলাই রোববার ব্যাংকের লেনদেন বন্ধ ছিল          5
2   ২২ ক্যারেট রুপার ভরি আগের ১ ৫১৬ টাকায় বিক্রি হবে          5
3   ২১ ১৮ ক্যারেট রুপার ভরি যথাক্রমে ১ ৪৩৫ ১ ২২৫ টাকা          5
4       সনাতন পদ্ধতির রুপার ভরি ৯৩৩ টাকায় বিক্রি হবে          5
5   পরের রোববার অর্থাৎ ১১ জুলাইও ব্যাংকের শাখা বন্...          5
6        ২০১১ সালে আউন্স সোনার দাম উঠেছিল ১ ৯২১ ডলারে          4
7   ১০ টাকা জরিমানা নির্ধারিত ভ্যাটের টাকার ওপর ২ ...          4
8                ২০১৩ সালের ২৭ জানুয়ারি সূচক হয়েছিল          4
9              তহবিলের আকার বেড়ে ৯ ১৮৮ টাকা দাঁড়ায়          3
10  পাঁচ মাসের কম সময়ের উড়োজাহাজ বিধ্বস্ত হয়ে ৩...          3
11                   শেষ ভরির দাম বেড়েছে ১২ ৩০৬ টাকা          3
12  মিনিটের আদ্দিস আবাবা ৬২ কিলোমিটার দূরে এক শহরে...          3
13  ১৯৭৫ সালে প্রথমবারের দেশের করদাতাদের ধরনে